In [ ]:
def calc_obs_props(theta):
    props = []
    ap_sp = simmer.build_sp(theta)
    [props.append(ap_sp.total_mag(band)) for band in ['DECam_g','DECam_r', 'DECam_z'] ]
    [props.append(ap_sp.sbf_mag(band)) for band in ['DECam_g','DECam_r', 'DECam_z'] ]
    return [np.array(props),]

with mpire.WorkerPool(n_jobs=8) as pool:
    obs_props_val = pool.map(calc_obs_props,[t for t in t_val.numpy()], progress_bar=True)

obs_props_val= np.array(obs_props_val).squeeze()
summ_stats = post.posterior_estimator._embedding_net(x_val)
summ_stats = summ_stats.detach().numpy()

mod_list = []
for j in range(summ_stats.shape[1]):
    model = pysr.PySRRegressor(
        binary_operators=["+", "-", "*", "/"], 
        unary_operators=["exp", "sqrt","square"], 
        variable_names = ['m_g','m_r','m_z','sbf_g','sbf_r','sbf_z']
    )
    model.fit(obs_props_val, summ_stats[:,j])
    mod_list.append(model)

for model in mod_list:
    print (model.get_best()['sympy_format'] )

fig, axes = plt.subplots(2,5, figsize = (20,7))
for i in range(summ_stats.shape[1]):
    summ_cur = summ_stats[:,i]
    pred_cur = mod_list[i].predict(obs_props_val)
    max_cur = np.max([np.max(summ_cur),np.max(pred_cur)])
    min_cur = np.min([np.min(summ_cur),np.min(pred_cur)])
    ax_cur = axes.flatten()[i]
    ax_cur.plot([min_cur,max_cur],[min_cur,max_cur], 'k--')
    ax_cur.plot(summ_cur, pred_cur, 'o' )
    ax_cur.set_xlim(min_cur,max_cur)
    ax_cur.set_ylim(min_cur,max_cur)
    ax_cur.set_title(rf'${mod_list[i].latex()}$')
plt.show()